In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import time
import struct
import os
from IPython.display import clear_output

In [2]:
#mt5.login(3081906, password="qllj3qcj", server = "Just2Trade-MT5")

In [3]:
# подключимся к MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    quit()

In [4]:
def take_history():
    History = mt5.history_deals_get(datetime.now() - timedelta(hours=19) , datetime.now(), group="PEP")
    return History

In [5]:
def take_history2():
    History = mt5.history_deals_get(datetime.now() - timedelta(hours=19) , datetime.now(), group="KO")
    return History

In [6]:
History_frame = pd.DataFrame(take_history())
History_frame2 = pd.DataFrame(take_history2())

In [7]:
History_frame

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,214320792,217392138,1616527963,1616527963867,0,0,123456,217392138,3,5.0,139.46,-0.02,0.0,0.0,0.0,PEP,"Открыли позицию,",
1,214320793,217392139,1616527963,1616527963941,1,1,123456,217392138,3,5.0,139.46,-0.02,0.0,0.0,0.0,PEP,"Открыли позицию,",
2,214320799,217392143,1616527978,1616527978168,0,0,123456,217392143,3,5.0,139.43,-0.02,0.0,0.0,0.0,PEP,"Открыли позицию,",


In [8]:
last_tick_KO_ask = mt5.symbol_info_tick("KO")

In [9]:
Sum = 0
for i in History_frame[13]:
    Sum = Sum + i
print(Sum)
Sum2 = 0
for i in History_frame2[13]:
    Sum2 = Sum2 + i
print(Sum2)

0.0
0.0


In [10]:
def take_rates(): 
    gbpusd_rates = mt5.copy_rates_from_pos("PEP", mt5.TIMEFRAME_M1, 0, 15)
    eurusd_rates = mt5.copy_rates_from_pos("KO", mt5.TIMEFRAME_M1, 0, 15)
    return gbpusd_rates, eurusd_rates

In [11]:
def calculation(rate1, rate2, lotko, lotpep):
    spread = rate1 - rate2 * lotko / lotpep # Вычисляем спред
    spread = np.array(spread)
    median = np.median(spread)
    Open_up = list()
    Open_down = list()
    Open_UP =0
    Open_DOWN = 0
    #Вычисляем верхнюю и нижнюю границу
    for i in range(0,len(spread)):
        if spread[i] > median:
            Open_up.append(spread[i])
        if spread[i] < median:
            Open_down.append(spread[i])
    Open_UP =np.median(Open_up)
    Open_DOWN = np.median(Open_down)
    return Open_UP, Open_DOWN, median

In [12]:
#Достаём значения для открытия и закрытия позиции
lotpep = 5.0
lotko = 13.0
values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], lotko, lotpep)
Open_Up = values[0]
Open_Down = values[1]
Close = values[2]
print(Open_Up, Open_Down, Close)
#Некоторые переменные для подсчитывания открытых позиции и прибыли
Pos_Up = False
Pos_Down = False
price_Pep_old = 0
price_Pep_new = 0
price_KO_old = 0
price_KO_new = 0
while(True):
    if datetime.now().minute % 15 == 0 and datetime.now().second == 0:
        if Pos_Up == True:
            Pos_Up = False
        if Pos_Down == True:
            Pos_Down = False
        values = calculation(take_rates()[0]['close'], take_rates()[1]['close'], lotko, lotpep)
        Open_Up = values[0]
        Open_Down = values[1]
        Close = values[2]
        print(Open_Up, Open_Down, Close)
        time.sleep(1)
    last_tick_KO_ask = mt5.symbol_info_tick("KO").ask
    last_tick_PEP_ask = mt5.symbol_info_tick("PEP").ask
    last_tick_KO_bid = mt5.symbol_info_tick("KO").bid
    last_tick_PEP_bid = mt5.symbol_info_tick("PEP").bid
    
    last_tick_KO_last = mt5.symbol_info_tick("KO").last
    last_tick_PEP_last = mt5.symbol_info_tick("PEP").last

#     if Pos_Up == False: #Купить дешевую продать дорогую
#         last_spread_up = last_tick_PEP_ask - last_tick_KO_bid * lotko / lotpep
        
#     if Pos_Down == False:#Купить дешевую продать дорогую
#         last_spread_down = last_tick_PEP_bid - last_tick_KO_ask * lotko / lotpep
        
#     if Pos_Up == True: #Купить дешевую продать дорогую
#         last_spread_up = last_tick_PEP_bid - last_tick_KO_ask * lotko / lotpep
        
#     if Pos_Down == True:#Купить дешевую продать дорогую
#         last_spread_down = last_tick_PEP_ask - last_tick_KO_bid * lotko / lotpep
    
#     if last_spread_up > Open_Up and Pos_Up == False:
#         Pos_Up = True   

#     if last_spread_down < Open_Down and Pos_Down == False:
#         Pos_Down = True
        
#     if last_spread_up <= Close and Pos_Up == True:
#         Pos_Up = False
        
#     if last_spread_down >= Close and Pos_Down == True:
#         Pos_Down = False
        
    last_spread = last_tick_PEP_last - last_tick_KO_last * lotko / lotpep
    if last_spread > Open_Up and Pos_Up == False:
        Pos_Up = True
        price_Pep_old = last_tick_PEP_last
        price_KO_old = last_tick_KO_last

    if last_spread < Open_Down and Pos_Down == False:
        Pos_Down = True
        price_Pep_old = last_tick_PEP_last
        price_KO_old = last_tick_KO_last

    if (price_Pep_old - last_tick_PEP_last + last_tick_KO_last * lotko / lotpep - price_KO_old * lotko / lotpep > 0.1) and Pos_Up == True:
        Pos_Up = False
        
    if (last_tick_PEP_last - price_Pep_old - last_tick_KO_last * lotko / lotpep + price_KO_old * lotko / lotpep > 0.1) and Pos_Down == True:
        Pos_Down = False
    print('last_tick_PEP_ask ', last_tick_PEP_ask)
    print('last_tick_PEP_bid ', last_tick_PEP_bid)
    print('last_tick_KO_ask ', last_tick_KO_ask)
    print('last_tick_KO_bid ', last_tick_KO_bid)
#    print('Прибыль ', 5 * (last_tick_PEP_last - price_Pep_old ))
    print('last_tick_PEP_last ', last_tick_PEP_last)
    print('last_tick_KO_last', last_tick_KO_last)
    print('last_tick_KO_last * lotko / lotpep', last_tick_KO_last * lotko / lotpep)
    print('last_spread', last_spread)
    print('lotko', lotko)
    print('lotpep ', lotpep)
    print('Open_Up ', Open_Up)
    print('Open_Down ',Open_Down)
    print('Close ',Close)
    print('Close + (Open_Up - Close) / 2 ',Close + (Open_Up - Close) / 2)
    print('Close - (Close - Open_Down) / 2 ',Close - (Close - Open_Down) / 2)
    print("Pos_Up ",Pos_Up)
    print('Pos_Down ',Pos_Down)
#     print('last_tick_PEP_bid-last_tick_KO_ask * lotko / lotpep', last_tick_PEP_bid - last_tick_KO_ask * lotko / lotpep)
#     print('last_tick_PEP_ask-last_tick_KO_bid * lotko / lotpep',last_tick_PEP_ask - last_tick_KO_bid * lotko / lotpep)
#     print('last_spread_up ',last_spread_up)
#     print('last_spread_down ',last_spread_down)
    clear_output(wait=True)

last_tick_PEP_ask  140.2
last_tick_PEP_bid  135.89
last_tick_KO_ask  51.46
last_tick_KO_bid  51.33
last_tick_PEP_last  139.46
last_tick_KO_last 51.39
last_tick_KO_last * lotko / lotpep 133.614
last_spread 5.846000000000004
lotko 13.0
lotpep  5.0
Open_Up  5.956000000000017
Open_Down  5.895999999999987
Close  5.941999999999979
Close + (Open_Up - Close) / 2  5.948999999999998
Close - (Close - Open_Down) / 2  5.918999999999983
Pos_Up  False
Pos_Down  

KeyboardInterrupt: 